# Анализ характеристик, влияющих на успеваемость студентов


In [3]:
import pandas as pd
import numpy as np

# Отключаем предупреждения
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None

In [5]:
# Загрузка данных
train_df = pd.read_csv('train.csv')
questions_df = pd.read_csv('questions.csv')
lectures_df = pd.read_csv('lectures.csv')

In [7]:
# Отбираем только вопросы
questions_only = train_df[train_df['content_type_id'] == 0].copy()

In [9]:
print('Размерности датасетов:')
print(f'train_df: {train_df.shape}')
print(f'questions_df: {questions_df.shape}')
print(f'lectures_df: {lectures_df.shape}')

Размерности датасетов:
train_df: (101230332, 10)
questions_df: (13523, 5)
lectures_df: (418, 4)


## 1. Анализ времени решения

In [21]:
# Анализ времени решения
time_analysis = questions_only.groupby('answered_correctly')['prior_question_elapsed_time'].agg([
    'count', 'mean', 'median', 'std'
]).round(2)

print('Статистика времени решения по правильности ответа:')
print(time_analysis)



Статистика времени решения по правильности ответа:
                       count      mean   median       std
answered_correctly                                       
0                   33901107  25641.99  21000.0  20620.63
1                   64977687  25309.98  21000.0  19587.16


## 2. Анализ интервалов между вопросами

In [23]:
# Анализ интервалов между вопросами
questions_only['time_between'] = questions_only.groupby('user_id')['timestamp'].diff()
time_intervals = questions_only.groupby('answered_correctly')['time_between'].agg([
    'count', 'mean', 'median', 'std'
]).round(2)

print('\nСтатистика интервалов между вопросами:')
print(time_intervals)


Статистика интервалов между вопросами:
                       count         mean   median           std
answered_correctly                                              
0                   33900566  25177173.93  39769.0  5.546770e+08
1                   64977078  17971423.31  32587.0  4.632076e+08


## 3. Анализ влияния объяснений

In [26]:
# Анализ влияния объяснений
explanation_impact = questions_only.groupby('prior_question_had_explanation')['answered_correctly'].agg([
    'count', 'mean', 'std'
]).round(3)

print('Влияние наличия объяснений на успеваемость:')
print(explanation_impact)

Влияние наличия объяснений на успеваемость:
                                   count   mean    std
prior_question_had_explanation                        
False                            9193234  0.501  0.500
True                            89685560  0.673  0.469


## 4. Анализ групп вопросов

In [29]:
# Анализ групп вопросов
container_analysis = questions_only.groupby('task_container_id').agg({
    'answered_correctly': ['count', 'mean', 'std'],
    'prior_question_elapsed_time': 'mean'
}).round(3)

print('\nСтатистика по группам вопросов:')
print(container_analysis.describe())


Статистика по группам вопросов:
      answered_correctly                              \
                   count          mean           std   
count       10000.000000  10000.000000  10000.000000   
mean         9927.130000      0.713444      0.451345   
std         32457.455136      0.028732      0.013420   
min           169.000000      0.315000      0.370000   
25%           419.750000      0.695000      0.445000   
50%          1308.500000      0.710000      0.454000   
75%          5706.750000      0.729000      0.460000   
max        801910.000000      0.838000      0.500000   

      prior_question_elapsed_time  
                             mean  
count                10000.000000  
mean                 24213.716582  
std                   1619.597572  
min                  17981.387000  
25%                  23132.166500  
50%                  24485.039500  
75%                  25446.127000  
max                  32470.996000  


## 5. Анализ последовательностей ответов

In [9]:
# Анализ последовательностей ответов
questions_only['prev_correct'] = questions_only.groupby('user_id')['answered_correctly'].shift(1)
questions_only['prev_time'] = questions_only.groupby('user_id')['prior_question_elapsed_time'].shift(1)

sequence_analysis = questions_only.groupby('prev_correct')['answered_correctly'].agg([
    'count', 'mean', 'std'
]).round(3)

print('Влияние предыдущего ответа на успеваемость:')
print(sequence_analysis)

Влияние предыдущего ответа на успеваемость:
                 count   mean    std
prev_correct                        
0.0           33827647  0.587  0.492
1.0           65049997  0.694  0.461


## 6. Анализ по разделам

In [11]:
# Объединяем с метаданными вопросов
questions_meta = questions_only.merge(questions_df, 
                                     left_on='content_id',
                                     right_on='question_id',
                                     how='left')

# Анализ по разделам
part_analysis = questions_meta.groupby('part')['answered_correctly'].agg([
    'count', 'mean', 'std'
]).round(3)

print('Статистика по разделам:')
print(part_analysis)

Статистика по разделам:
         count   mean    std
part                        
1      7454570  0.745  0.436
2     18743404  0.709  0.454
3      8639907  0.701  0.458
4      8067676  0.631  0.483
5     40908153  0.610  0.488
6     10501472  0.669  0.470
7      4956118  0.660  0.474


## 7. Анализ тегов

Запустите один из вариантов в зависимости от наличия времени и ресурсов компьютера

### Анализ тегов на ограниченной выборке (обрабатывает ограниченное количество данных, отрабатывает быстро)

In [28]:
# Анализ тегов на ограниченной выборке
def analyze_tags_sample(questions_meta, sample_size=10000):
    # Берем случайную выборку
    sample_df = questions_meta.sample(n=sample_size, random_state=42)
    
    # Анализируем теги
    tag_stats = {}
    
    for _, row in sample_df.iterrows():
        if isinstance(row['tags'], str):
            tags = row['tags'].split()
            for tag in tags:
                if tag not in tag_stats:
                    tag_stats[tag] = {'correct': 0, 'total': 0}
                tag_stats[tag]['total'] += 1
                if row['answered_correctly'] == 1:
                    tag_stats[tag]['correct'] += 1
    
    # Преобразуем в DataFrame
    tag_df = pd.DataFrame([
        {
            'tag': tag,
            'success_rate': data['correct'] / data['total'],
            'total_questions': data['total']
        }
        for tag, data in tag_stats.items()
    ])
    
    # Фильтруем теги с малым количеством вопросов
    tag_df = tag_df[tag_df['total_questions'] >= 10]
    
    return tag_df

# Вариант 1: Анализ на выборке
tag_analysis = analyze_tags_sample(questions_meta, sample_size=10000)

### Анализ тегов по частям (обрабатывает все данные, отрабатывает долго)

In [ ]:
# Анализ тегов по частям
def analyze_tags_in_batches(questions_meta, batch_size=5000):
    total_tags = {}
    
    # Обрабатываем данные батчами
    for start in range(0, len(questions_meta), batch_size):
        end = start + batch_size
        batch = questions_meta.iloc[start:end]
        
        # Анализируем батч
        for _, row in batch.iterrows():
            if isinstance(row['tags'], str):
                tags = row['tags'].split()
                for tag in tags:
                    if tag not in total_tags:
                        total_tags[tag] = {'correct': 0, 'total': 0}
                    total_tags[tag]['total'] += 1
                    if row['answered_correctly'] == 1:
                        total_tags[tag]['correct'] += 1
                        
        # Промежуточный отчет
        print(f"Обработано {end} из {len(questions_meta)} записей")
    
    # Преобразуем результаты в DataFrame
    tag_df = pd.DataFrame([
        {
            'tag': tag,
            'success_rate': data['correct'] / data['total'],
            'total_questions': data['total']
        }
        for tag, data in total_tags.items()
    ])
    
    return tag_df[tag_df['total_questions'] >= 10]

# или Вариант 2: Анализ по частям
tag_analysis = analyze_tags_in_batches(questions_meta, batch_size=500000)

### Вывод результатов

In [30]:
# Вывод результатов
print('Топ-5 самых сложных тегов:')
print(tag_analysis.nsmallest(5, 'success_rate')[['tag', 'success_rate', 'total_questions']])

print('\nТоп-5 самых легких тегов:')
print(tag_analysis.nlargest(5, 'success_rate')[['tag', 'success_rate', 'total_questions']])

Топ-5 самых сложных тегов:
     tag  success_rate  total_questions
16   151      0.389831               59
50    24      0.413793               29
163   58      0.437500               32
135  161      0.438776               98
107   78      0.444444               18

Топ-5 самых легких тегов:
     tag  success_rate  total_questions
173   40      0.944444               18
151  177      0.933333               30
46   149      0.918919               37
166  187      0.913043               23
93    13      0.894737               19


## 8. Влияние лекций

In [33]:
# Анализ влияния лекций
def calculate_lecture_impact(user_data):
    user_data = user_data.sort_values('timestamp')
    user_data['lectures_viewed'] = (user_data['content_type_id'] == 1).cumsum()
    return user_data[user_data['content_type_id'] == 0]

lecture_impact = pd.concat([calculate_lecture_impact(group) 
                           for _, group in train_df.groupby('user_id')])

lecture_stats = lecture_impact.groupby('lectures_viewed')['answered_correctly'].agg([
    'count', 'mean', 'std'
]).round(3)

print('Влияние количества просмотренных лекций:')
print(lecture_stats)

Влияние количества просмотренных лекций:
                    count   mean    std
lectures_viewed                        
0                23452220  0.596  0.491
1                 6270189  0.667  0.471
2                 4699417  0.666  0.472
3                 3703144  0.669  0.471
4                 3240789  0.670  0.470
...                   ...    ...    ...
393                    55  0.600  0.494
394                  1815  0.646  0.478
395                  2331  0.850  0.357
396                    15  0.933  0.258
397                   529  0.783  0.413

[398 rows x 3 columns]


## 6. Анализ вляния типов лекций

In [35]:
# Объединяем данные о лекциях
lectures_meta = train_df.merge(lectures_df,
                              left_on='content_id',
                              right_on='lecture_id',
                              how='inner')

# Анализ эффективности разных типов лекций
lecture_type_impact = lectures_meta.groupby('type_of').agg({
    'user_id': 'count',  # количество просмотров
    'prior_question_had_explanation': 'mean'  # влияние на последующие объяснения
}).round(3)

print('Статистика по типам лекций:')
print(lecture_type_impact)

Статистика по типам лекций:
                  user_id prior_question_had_explanation
type_of                                                 
concept           2195409                       0.328385
intention          145066                       0.256428
solving question   917263                       0.498889
starter              5013                        0.97346
